In [ ]:
from fastai.vision import *
import fastai
import pandas as pd

This notebook is based on `fast.ai` course `v3` (2019) and version of `fastai` `1.0.57`. It looks like it changes pretty significantly from version to version so many 1-year old notebooks are pretty outdated. 

It covers just the lesson 1: 
- train the model with the default `lr` for a few epochs with image `size=224`;
- find new `lr` and unfreeze all the layers; 

In [ ]:
fastai.__version__

## data preparation

There are a few issues with the file system:

- your files are stored in `../input/dog-breed-identification`, not in `../input/`;
- during different runs of this notebook you may encounter a bug and `len(path_test.ls())` will give you `10358` instead of `10357`; in this case you have to specify `path_test = path / 'test/test'` and `test='test/test'`;
- finally, you should store the model in `tmp` directory specified below and `'submission.csv'` in the current directory;

In [ ]:
Path('../input').ls(), 

In [ ]:
path = Path('../input/dog-breed-identification')
path_train = path / 'train'
path_test = path / 'test/test'
path_model = Path('/tmp/model/')

In [ ]:
path_train, path_test

In [ ]:
path.ls()

In [ ]:
len(path_train.ls()), len(path_test.ls())

In [ ]:
list(Path('../input/dog-breed-identification').glob('**'))

In [ ]:
bs, size = 64, 224

In [ ]:
data = ImageDataBunch.from_csv(path=path,
                               folder='train',
                               csv_labels='labels.csv',
                               ds_tfms=get_transforms(),
                               suffix='.jpg',
                               test='test/test',
                               size=size,
                               bs=bs,
                               num_workers=0).normalize(imagenet_stats)

In [ ]:
len(data.train_ds), len(data.valid_ds), len(data.test_ds.items)

In [ ]:
8178 + 2044

> ## vanilla model

In [ ]:
learn = cnn_learner(data, 
                    base_arch=models.resnet50, 
                    metrics=accuracy, 
                    model_dir=path_model)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

So we have to choose a point with the steepest descent where the plot goes down for a while. That's `1e-3` or basically the same as the default `lr`. So we may just train with the default rate.

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn.save('stage-1-50-224')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

Here we don't have any steep curves so we may just choose `1e-6` or `1e-5` as the left limit and we choose `1e-4` as the right limit (we may use the rule that it's equal to previous `lr // 10` or we may just use the standard value for it which is `1e-4` as well).

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-6,1e-4))

So no improvement, let's load the previous model.

In [ ]:
learn.load('stage-1-50-224');

## submission

In [ ]:
preds, _ = learn.get_preds(DatasetType.Test)

In [ ]:
data.test_ds.items[0]

In [ ]:
path_test

In [ ]:
def get_numeric_part(file_path, path_test):
    low = len(str(path_test)) + 1
    return str(file_path)[low:-4]

In [ ]:
get_numeric_part(data.test_ds.items[0], path_test)

In [ ]:
df = pd.DataFrame(preds.numpy(), columns=data.classes)
filenames = [get_numeric_part(fp, path_test) for fp in data.test_ds.items]
df.insert(0, "id", filenames)
df_sorted = df.sort_values(by='id')

In [ ]:
df_sorted.head()

In [ ]:
df_sorted.iloc[0, 1:].sum()

In [ ]:
df_sorted.to_csv('submission.csv', index=False)

In [ ]:
ls